# RSA

we will generate 2 random primes using the fonction RandomPrime , this fonctions generate randomly numbers in range of the "size" and check if it's a prime or not


In [6]:
from crypto import RandomPrime
size_bits = 16

p = RandomPrime(size_bits,m=40)
q = RandomPrime(size_bits,m=40)
while p==q: 
    q = RandomPrime(size_bits,m=40)

    
print(p)
print(q)

54751
64663


Then we will calcule (p-1) x (q-1) which is the Euler totien 




Euler totien of N is the number of integers that are comprime with N up to N


2 integers a and b are **Comprime** if the pgcd(a,b) = 1

The Euler totien of a prime number p is *p-1* because because all the numbers below p are **comprime** to it

In [12]:
N = p*q
phi = (p-1)*(q-1)

print(N)
print(phi)

3540363913
3540244500


Now we will find a randome "e" smaller than phi(N) such that pgcd(e, phi(N)) = 1 

In [21]:
from crypto import xgcd
from random import randrange

In [24]:
e = randrange(phi)
gcd = xgcd(e,phi)[0]
while gcd !=1:
    e = randrange(phi)
    gcd = xgcd(e,phi)[0]
print(e)

3330190943


now that we have find "e" that much the condition we will calculate the "d" which is the invese of e modulo phi

In [30]:
from crypto import InverseMod

In [34]:
d = InverseMod(e,phi)

In [36]:
print(f"p = {p}")
print(f"q = {q}")
print(f"N = {N}")
print(f"phi = {phi}")
print(f"e = {e}")
print(f"d = {d}")

p = 54751
q = 64663
N = 3540363913
phi = 3540244500
e = 3330190943
d = 2964670307


The tuple (N,e) is The public key and the tuple (N,d) is the private key

## Encryption


for a message "m" the encryption fonctions is : c = m^e mod N , where c is the cipher text

In [49]:
message = randrange(0, N-1)
cipher_text = pow(message, e, N)

print(f"message: {message}")
print(f"ciphertext: {cipher_text}")

message: 2521889450
ciphertext: 1220390330


## Decryption

The decryption of the ciphertext can be done using only the private key ($N$, $d$):

$$m=c^{d} \textit{mod N}$$

We can check that the equation gives back the original message:

$$c^{d} \textit{mod N}=m^{d*e}\textit{mod N}=m \textit{mod N}$$



In [53]:
plain_text = pow(cipher_text,d,N)
assert plain_text == message , "the recovred message is wrong ! check the algo"
print("the recovred message is : ",plain_text)

the recovred message is :  2521889450


In [92]:
def RsaKeyGenerator(size): 
    # the fonction returns a tuple of the public and private keys 
    p = RandomPrime(size)
    q = RandomPrime(size)
    while q==p : 
        q = RandomPrime(size)
        
    N  = p*q
    phi = (p-1) * (q-1)
    e = randrange(phi)
    gcd = xgcd(e,phi)[0]
    while gcd !=1:
        e = randrange(phi)
        gcd = xgcd(e,phi)[0]
        
    d = InverseMod(e,phi)
    return (N,e),(N,d)
def RsaEncrypt(message,public_key):
    N,e = public_key[0],public_key[1]
    return pow(message,e,N)
def RsaDecrypt(message,private_key):
    N,d = private_key[0],private_key[1]
    return pow(message,d,N)


In [93]:
# we are assuming that we have a comunication between 2 parties bob and alice 
# so let's generate a public and private key for bob
key_size = 128 
public_key , private_key = RsaKeyGenerator(key_size)
print("public key : " ,public_key)
print("\n")
print("private key : ",private_key)

public key :  (81430036320771684030809881215801340700380965495309284827365794605778601137219, 1888411007589028357783387897713499936847613082671376232130615090020669473529)


private key :  (81430036320771684030809881215801340700380965495309284827365794605778601137219, 5004703469580397649282711069359255133749549589646883273041474471307740992849)


In [94]:
# now alice want to send a message to bob encrypted using his public key
message = b"Hey bob ! "
print("message :",message)
# we must verify if the length of the message is smaller than N
assert 8*len(message)< key_size,'The message is too long'



message : b'Hey bob ! '


In [95]:
# let's convert the bytes into an int and encrypt it using the public key 
msg_int = int.from_bytes(message,"big")
print(msg_int)

c_int = RsaEncrypt(msg_int,public_key)
print(c_int)

341882236002367491285280
46363621039738457134373275428553191035351261586740871118190429164178804236506


In [96]:
# now bob receive the cipher text and try to decrypt it using his private key
recovred_message_int = RsaDecrypt(c_int,private_key)
print(recovred_message_int)
assert msg_int == recovred_message_int,"The recovred message is wrong"

341882236002367491285280


In [98]:
recovred_message = recovred_message_int.to_bytes(len(message),'big')

In [99]:
recovred_message

b'Hey bob ! '

## Trying to break the RSA 

we are going to calculate the p and q from N to calculate the phi(n) and for this we are going to factorize the N 

In [101]:
key_size = 16
public_key , private_key = RsaKeyGenerator(key_size)
print("public key : ",public_key)
print("private_key :",private_key)

public key :  (2534129573, 1220524085)
private_key : (2534129573, 1956276401)


we are going to use the Pollard Rho algorithme fo factorize the N

In [102]:
def PollardRho(N):
    if N==1: 
        return N
    if N%2 ==0:
        return 2
    x = randrange(2,N)
    y = x 
    c = randrange(0,2)
    d =1 
    while d==1:
        x = (pow(x,2,N) + c + N)%N
        y = (pow(y,2,N) + c + N)%N
        y = (pow(y,2,N) + c + N)%N  
        d ,_,_ = xgcd(abs(x-y),N)
        if d==N:
            PollardRho(N)
    return d

In [133]:
N = public_key [0]
p2 = PollardRho(N)
q2 = N//p2
print(N,p2*q2)
print(f"N: {N}, p2 : {p2} , q2 : {q2} ")

2534129573 2534129573
N: 2534129573, p2 : 46567 , q2 : 54419 


now that we have calculated the p and q we can calculate the phi(n) = (p-1)*(q-1) and then calculate the d

For a big N we can not easaly calculate the p and q using PollarRho and for that there is some other algos that are more faster thant the PollarRho

# Paillier Crypto system

## Key Generation <a class="anchor" id="keygeneration"></a>

* Generate two random primes $p$ and $q$
* Calculate $N$, the product of $p$ and $q$
* if $N$ and $\phi(N)$ have common factors, go back to first step and generate new $p$ and $q$
* if $N$ and $\phi(N)$ don't share common factors (i.e. gcd is 1) then:
    * calculate $l$, the least common multiple of $p-1$ and $q-1$
    * calculate $N^2$
    * draw a random number $g$ in between 1 and $N^2$
    * calculate $\mu$ as the inverse of $L(g^l \textit{mod }N^2, N)$ in modulo $N$ where $L(x, n)=(x-1)/n$
    
The public key is ($N$, $g$) and the private key is ($N$, $l$, $\mu$) 

In [148]:
from crypto import RandomPrime, xgcd

In [140]:
# first we will find the prime numbers p and q 
size_bits = 16
p = RandomPrime(size_bits,60)
q = RandomPrime(size_bits,60)
while p==q : 
    q = RandomPrime(size_bits,60)
N = p*q
phi = (p-1) * (q-1)
gcd,_,_ = xgcd(N,phi)  
while gcd!=1:
    p = RandomPrime(size_bits,60)
    q = RandomPrime(size_bits,60)
    while p==q : 
        q = RandomPrime(size_bits,60)
    N = p*q
    phi = (p-1) * (q-1)
    gcd,_,_ = xgcd(N,phi) 

    
print(f"p = {p}")
print(f'q = {q}')
print(f"gcd(N,phi) = {gcd}")
    
    
#Note : if the 2 prime numbers p and q are in the same size in bits so the xgcd(N,phi ) is always 1  and that"s what we do in this exemple

p = 56911
q = 62549
gcd(N,phi) = 1


In [172]:
# now we are going to calculare the i, g  and the mu
from random import randrange
from crypto import InverseMod,LCM
# the fonction LCM calculate the Least Common Multiple of 2 integers , lcm(a,b) = a*b // gcd(a,b)

def L(x,n): 
    return int((x-1)/n)

l = LCM((q-1),(p-1))
g = randrange(1,N*N)

mu = InverseMod(L(pow(g,l,N*N),N),N)
mu = int(mu)

print(f"l = {l}")
print(f"g = {g}")
print(f"mu = {int(mu)}")





l = 1779803340
g = 7816644770407454975
mu = 1957643571


In [173]:
public_key = (N,g)
private_key = (N,l,mu)
print(f"public key is : {public_key}")
print(f"private key is : {private_key}")

public key is : (3559726139, 7816644770407454975)
private key is : (3559726139, 1779803340, 1957643571)


## Encryption Fonction


For the encryption we must find a random number r ;  r < N such it has no commun factor with N, then calculate c as : $$c = g^{m}*r^{N}(\text{mod }N^2)$$

In [179]:
message = randrange(N)
N , g = public_key


r = randrange(N)
gcd,_,_ = xgcd(r,N)
while gcd !=1:
    r = randrange(N)
    gcd,_,_ = xgcd(r,N)

    
cipher_text = (pow(g,message,N*N)*pow(r,N,N*N))% (N*N)

print(f'message : {m}')
print(f"r = {r}")
print(f"cipher text is : {cipher_text}")


message : 710858405
r = 2783801101
cipher text is : 7336294734005926846


## Decryption Fonction
Having the private key (N,l,mu) and the cipher c : 
 $$m = L(c^l(\text{mod }N^2), N)*\mu (\text{mod }N)$$

In [181]:
N , l , mu = private_key
recovred_message = (L(pow(cipher_text,l,N*N),N)* mu) % N
assert message == recovred_message , "The message and the recovred message are not the same"
print(f"the cipher text is : {cipher_text}")
print(f"the recovred text is : {recovred_message}")
print(f"the message is :{message} ")
      

the cipher text is : 7336294734005926846
the recovred text is : 492731126
the message is :492731126 


In [193]:
def Pailler_key_generator(size_bits):
    p = RandomPrime(size_bits,60)
    q = RandomPrime(size_bits,60)
    while p==q : 
        q = RandomPrime(size_bits,60)
    N = p*q
    phi = (p-1) * (q-1)
    gcd,_,_ = xgcd(N,phi)  
    while gcd!=1:
        p = RandomPrime(size_bits,60)
        q = RandomPrime(size_bits,60)
        while p==q : 
            q = RandomPrime(size_bits,60)
        N = p*q
        phi = (p-1) * (q-1)
        gcd,_,_ = xgcd(N,phi) 
        
    l = LCM((q-1),(p-1))
    g = randrange(1,N*N)

    mu = InverseMod(L(pow(g,l,N*N),N),N)
    mu = int(mu)

    return (N,g),(N,l,mu)


def Pailer_encrypt(message,public_key):
    N , g = public_key
    r = randrange(N)
    gcd,_,_ = xgcd(r,N)
    while gcd !=1:
        r = randrange(N)
        gcd,_,_ = xgcd(r,N)


    cipher_text = (pow(g,message,N*N)*pow(r,N,N*N))% (N*N)

    return cipher_text

def Pailler_decrypt(c, private_key): 
    N , l , mu = private_key
    m = (L(pow(c,l,N*N),N)* mu) % N
    return m
    

In [205]:
message = randrange(N)
public_key , private_key = Pailler_key_generator(32)
print(f"Public Key : {public_key}")
print(f"Private key : {private_key}")
c = Pailer_encrypt(message,public_key)
recovred_message = Pailler_decrypt(c,private_key)

print(f"The message is : {message} ")
print(f"The Cipher text is :{c} ")
print(f"The recovred message is : {recovred_message}")

Public Key : (11121031096768975457, 81887090803404941468193555859110239947)
Private key : (11121031096768975457, 347532221559692928, 2177723798879476234)
The message is : 1994252608 
The Cipher text is :6326711952016594520317857807096347795 
The recovred message is : 8140233049904590484


# El Gamal Cryptosystem

## Key Generation <a class="anchor" id="keygeneration"></a>

Steps:

* A trusted party chooses a prime number $p$ and an element of the field $g$ (of large order, ideally a generator).
* The party that generates the keys draws a private key $a$<$p$ and computes $A=g^a (\text{mod }p)$. Publishes $A$ as the public key

The public key is $A$ and the private key $a$.

In [208]:
from crypto import GeneratePrimeGeneratorPair

In [ ]:
size_bits = 16
p, g = GeneratePrimeGeneratorPair(size_bits)

print(p,g)

In [322]:
size_bits = 16
p, g = GeneratePrimeGeneratorPair(size_bits)

print(p,g)
from random import randrange
a = randrange(p)

A = pow(g,a,p)

print("A :",A)
print("a :",a)


38393 3693
A : 644
a : 30551


A is the public key and a is the private key

## Encryption fonction 


for the encryption, more than the public key A the sender must knows the p and g. We need to choose a random number k to compute the 2 parts of the cipher text (the cipher text is a tuple (c1 , c2) )

the key must change for every message sent, if the message is too long so we need to encrypt it with blocks so each block must to have his own k  

the encryption fonctions is : 
$$c_1=g^k (\text{mod }p)$$
$$c_2=mA^k (\text{mod }p)$$

In [317]:
message = randrange(p)

k = randrange(p)

c1 = pow(g,k,p)
c2 = message*pow(A,k,p) % p
c = (c1,c2)


print(f"message is : {message}")
print(f"k : {k}")
print(f"the cipher text is : {(c1,c2)}")




message is : 36222
k : 9179
the cipher text is : (17557, 23036)


 ## Decryption Fonction 
 
using the cipher text paire (c1 , c2) we can generate our message using the fonction below : 
$$m = (c_1^a)^{-1}*c_2 (\text{mod }p)$$

In [321]:
from crypto import InverseFermat
c1 , c2 = c
print(c1,c2)
# recovred_message = c2*InverseFermat(pow(c1,a,p),p) %p

recovred_message = c2*InverseFermat(pow(c1, a, p), p)%p
print(recovred_message)

17557 23036
36222


## Brut Force Attack of El Gamel

cracking El Gamel is same as resolving the Discret Algo Problem so if we can find the a we cracked the algorithm


In [358]:
size_bits = 16
p, g = GeneratePrimeGeneratorPair(size_bits)

print(p,g)
from random import randrange
a = randrange(p)

A = pow(g,a,p)

print("A :",A)
print("a :",a)


54983 8000
A : 11964
a : 52605


In [359]:
a_ = randrange(p)

while A!= pow(g,a_,p):
    a_ = randrange(p)
    
print(f"the private key is : {a}")
print(f"the cracked private key is : {a_}")


the private key is : 52605
the cracked private key is : 52605


for the size of 16 bit it seems easy but if we increase the size of bits we will not be able to crack it 

## Known cipher text attack 

alice want bob and charlie to send her messages , so she creates pair key (A,a) of El Gamal, so if charlie send to alice the cipher text and want her to decrypt it for him and send him the plain text, so alice must not decrypt the same cipher text twice because bob can intercept the cipher text and resend it to alice to decrypt it so he kan see what charlie wants to say.

Alice is forbiden to decrypt the same cipher text twice





Let's say Bob intercepted a ciphertext $c=(c_1, c_2)=(g^k, mA^k)$ that Charlie sent to Alice and he wants to know what Charlie has to say to Alice (get $m$). He can't send that ciphertext to Alice for decryption as Alice will know that Bob is trying to cheat. But he can do the following...


Bob only knows the ciphertext $c_1$ and $c_1$ and creates a random message and a ephemeral key $m^{\prime}$, $k^{\prime}$ and computes the following ciphertext to send to Alice for decryption:

$$c^{\prime}=(c_1g^{k^{\prime}}, c_2A^{k^{\prime}}m^{\prime})$$

Alice will decrypt this, since this ciphertext $c^{\prime}$ is different from the original ciphertext $c$. Call the decrypted ciphertext $m^{\prime\prime}$. This is what Bob gets as the decryption of $c^{\prime}$ from Alice.

At this point Bob knows $c^{\prime}=(c_1g^{k^{\prime}}, c_2A^{k^{\prime}}m)$, the plaintext for $c^{\prime}$, i.e $m^{\prime\prime}$ and obviously the public key $A$. 

$$c^{\prime}=(c_1g^{k^{\prime}}, c_2A^{k^{\prime}}m^{\prime})=(g^{k+k^{\prime}},mm^{\prime}A^{k+k^{\prime}})$$

Therefore he determines that

$$m^{\prime\prime}=m*m^{\prime} (\text{mod }p)$$

or equivalently:

$$m=m^{\prime\prime}*(m^{\prime})^{-1} (\text{mod }p)$$

In [361]:
size_bits = 256
p, g = GeneratePrimeGeneratorPair(size_bits)



a = randrange(p)
A = pow(g,a,p)

print(f"p: {p}")
print(f"g: {g}")
print(f"A (public key): {A}")
print(f"a (private key): {a}")

p: 111157524683398064813452585815223631535230942814590106810025157490704558817187
g: 72956700057268738816717151712816626185122925583294407884886624965285750695906
A (public key): 66255688300942333646230586606157280857996066822689478063176011396619194391442
a (private key): 39514830202202660210210181580532439600421727837872610249199676912420128695094


In [362]:
message = b'Charlie ==> Alice'
assert 8 * len(message) <= size_bits , 'Message too long'
print(f"message charlie to alice is : {message}")

msg_int = int.from_bytes(message,"big")
print(msg_int)

message charlie to alice is : b'Charlie ==> Alice'
22937664268843740728723453030668863890277


In [363]:
#now charlie will encrypt the message 
k = randrange(p)
c1 = pow(g,k,p)
c2 = msg_int * pow(A,k,p) %p

c = (c1,c2)

print(f"message is : {message}")
print(f"message in integer form is : {msg_int}")
print(f"c  : {c}")
print(f"m  : {m}")


message is : b'Charlie ==> Alice'
message in integer form is : 22937664268843740728723453030668863890277
c  : (47473183675759739077048721278407996507770010600229783614456698746419156104074, 67185400036348664350217143457329013552856831608112203028526835083753562256363)
m  : 25429


Bob intercept the message from Charlie , he can't send it to Alice because she can not decrypt the same message twice so he will calculate new k' and m'

In [364]:
k_ = randrange(p)
m_ = randrange(p)

c_ = (c1*pow(g,k_,p),c2*pow(A,k_,p)*m_)
print(c_)

(4516560000467644442926293200719512164958046897842282190610977537090646347681704119927896806799486044246398965451862336789258329195040194080275988881356228, 127269844424887953905741646612135579852904369422247837545906971255517655084434968306754405549471217824660546472973112140096176970214990199946596330122041182633648989781809338977452540698949853829038307673221215256093702516799477840)


In [365]:
# now he will send c_ to alice to decrypt it 
c1 , c2 = c_
m__ = c2*InverseFermat(pow(c1, a, p), p)%p
print(m__)

71588515719200505506062159811498171266387536840203129643076805061545182405971


In [366]:
# now Bob has the recovred message m'' so he can find the m by applying the inverse 

inverse_m_ = InverseFermat(m_,p) 
recovred_m = m__ * inverse_m_ % p 
print("m : ",recovred_m)

recovred_message = recovred_m.to_bytes(len(message),'big')
print(recovred_message)

m :  22937664268843740728723453030668863890277
b'Charlie ==> Alice'


So we find the same message that Charlie sends to Alice and so we cracked the El Gamal Algorithm successfuly 